In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
# import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import math
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow import keras

Using TensorFlow backend.


In [2]:
from random import randint
from sklearn.model_selection import train_test_split

In [19]:


train_count = 5000
test_count = 10
max_sent_length = 100
embedding_size = 300

X = np.random.randn(train_count, max_sent_length, embedding_size)
Y = np.zeros((train_count, 1))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

for i in range(x_train.shape[0]):
    y_train[i:] = randint(0, 5)
for i in range(x_test.shape[0]):
    y_test[i:] = randint(0, 5)


In [48]:
train_count = 50
test_count = 10
max_sent_length = 100
embedding_size = 300
label_head = [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
]
label_dim = len(label_head[0])
X = np.random.randn(train_count, max_sent_length, embedding_size)
Y = np.zeros((train_count, label_dim))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

for i in range(x_train.shape[0]):
    y_train[i:] = label_head[randint(0, (label_dim - 1))]
for i in range(x_test.shape[0]):
    y_test[i:] = label_head[randint(0, (label_dim - 1))]

In [195]:
def getModel(input_shape):
    X_input = keras.layers.Input(shape=input_shape)
    X = keras.layers.Lambda(lambda x: K.mean(x, axis=1, keepdims=True))(X_input)
    X = keras.layers.Flatten()(X)
    X = keras.layers.Dense(128, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(256, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(128, activation=tf.nn.relu)(X)
    X = keras.layers.Dense(label_dim, activation=tf.nn.sigmoid)(X)
    model = keras.Model(inputs=X_input, outputs=X, name='HappyModel')
    return model


In [196]:
model = getModel((max_sent_length, embedding_size))
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [197]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 100, 300)          0         
_________________________________________________________________
lambda_10 (Lambda)           (None, 1, 300)            0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_146 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_147 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_148 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_149 (Dense)            (None, 5)                 645       
Total para

In [203]:
model.fit(x=x_train, y=y_train, epochs=30)

Epoch 1/30
40/40 [==============================] - 0s 475us/step - loss: 0.2225 - acc: 0.9450
Epoch 2/30
40/40 [==============================] - 0s 404us/step - loss: 0.1938 - acc: 0.9650
Epoch 3/30
40/40 [==============================] - 0s 388us/step - loss: 0.1663 - acc: 0.9650
Epoch 4/30
40/40 [==============================] - 0s 379us/step - loss: 0.1404 - acc: 0.9650
Epoch 5/30
40/40 [==============================] - 0s 459us/step - loss: 0.1170 - acc: 0.9700
Epoch 6/30
40/40 [==============================] - 0s 437us/step - loss: 0.0974 - acc: 0.9850
Epoch 7/30
40/40 [==============================] - 0s 432us/step - loss: 0.0799 - acc: 1.0000
Epoch 8/30
40/40 [==============================] - 0s 540us/step - loss: 0.0651 - acc: 1.0000
Epoch 9/30
40/40 [==============================] - 0s 458us/step - loss: 0.0530 - acc: 1.0000
Epoch 10/30
40/40 [==============================] - 0s 490us/step - loss: 0.0427 - acc: 1.0000
Epoch 11/30
40/40 [==============================

In [204]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Loss = %.2f  and test accuracy = %.2f' % (test_loss, test_acc))

10/10 [==============================] - 0s 657us/step
Loss = 0.63  and test accuracy = 0.82


In [205]:
model.predict(x_test)

array([[2.7706863e-02, 8.1126634e-03, 5.0544575e-02, 4.6070855e-02,
        1.8722810e-02],
       [9.9107973e-02, 5.8501951e-02, 2.9560188e-03, 1.2464639e-01,
        1.6373907e-01],
       [6.1136480e-02, 2.1559885e-02, 6.6424310e-02, 1.3963640e-01,
        8.9472450e-02],
       [7.8938585e-03, 1.3653224e-03, 1.6141546e-01, 2.9851133e-01,
        8.7973073e-02],
       [1.5549799e-02, 9.1083091e-01, 7.0839003e-03, 2.7544316e-02,
        1.7734746e-02],
       [3.4940448e-02, 1.1669303e-03, 1.5184385e-01, 2.3161526e-01,
        3.1316010e-03],
       [9.1959909e-02, 4.8002210e-02, 1.5520123e-01, 2.9238725e-02,
        9.9846959e-02],
       [1.2991412e-01, 7.6283081e-03, 5.4809773e-01, 1.3544178e-01,
        4.3616220e-03],
       [1.4085225e-02, 4.6511908e-04, 9.7253859e-02, 4.9876934e-01,
        2.2728758e-01],
       [1.2491641e-02, 4.4266723e-02, 4.8149689e-03, 4.6826418e-02,
        6.0761929e-01]], dtype=float32)

In [201]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])